In [11]:
import pandas as pd
import numpy as np
from time import time
import math

from plot.particle_swarm import ParticleSwarm
from plot.bounded_function import schaffer2, ripple25, schaffer1, bukin

In [12]:
def build_stats_pso(objective_function, TOTAL_RUNS = 20, CLOSE_UMBRAL = 20):
    actual_minimum = objective_function.glob_min[1]
    actual_optimum = objective_function.glob_min[0]

    x1_range = objective_function.range_x1
    x2_range = objective_function.range_x2

    durations = []
    minimum_errors = []
    optimum_errors = []

    for i in range(TOTAL_RUNS):
        pt = ParticleSwarm(100, x1_range, x2_range)
        pt.initialize_points()
        pt.update_points(objective_function)
        values_obtained: list[float] = []
        start = time()

        values_obtained = []
        avg = 0
        for i in range(CLOSE_UMBRAL):
            pt.update_points(objective_function)
            values_obtained.append(pt.best_value)
            avg += pt.best_value
        avg /= CLOSE_UMBRAL
        while True:
            pt.update_points(objective_function)
            avg *= CLOSE_UMBRAL
            avg -= values_obtained[-CLOSE_UMBRAL]
            values_obtained.append(pt.best_value)
            avg += values_obtained[-1]
            avg /= CLOSE_UMBRAL
            if abs(avg - values_obtained[-1]) < 1e-8:
                break

        end = time()

        minimum = pt.best_value
        optimum = pt.best_pos

        duration = end - start
        minimum_error = abs(minimum - actual_minimum)
        optimum_error = np.linalg.norm(np.array(optimum) - np.array(actual_optimum))

        durations.append(duration)
        minimum_errors.append(minimum_error)
        optimum_errors.append(optimum_error)

    median_duration = np.median(durations)
    mean_duration = np.mean(durations)
    std_duration = np.std(durations)

    median_minimum_error = np.median(minimum_errors)
    mean_minimum_error = np.mean(minimum_errors)
    std_minimum_error = np.std(minimum_errors)

    median_optimum = np.median(optimum_errors, axis=0)
    mean_optimum = np.mean(optimum_errors, axis=0)
    std_optimum = np.std(optimum_errors, axis=0)

    summary = pd.DataFrame(
        {
            "Promedio": [mean_duration, mean_minimum_error, median_optimum],
            "Mediana": [median_duration, median_minimum_error, mean_optimum],
            "Desv. Est.": [std_duration, std_minimum_error, std_optimum],
        },
        index=["Duración", "Error c.r. Mínimo", "Error c.r. Óptimo"],
    )
    markdown_table = summary.to_markdown(index=True)

    # Print the Markdown table
    print(f"Particle Swarm Optimization with function {objective_function.name} and {TOTAL_RUNS} runs")
    print(markdown_table)

In [13]:
summary = build_stats_pso(bukin, 100, 40)


Particle Swarm Optimization with function Bukin and 100 runs
|                   |   Promedio |   Mediana |   Desv. Est. |
|:------------------|-----------:|----------:|-------------:|
| Duración          |  0.379757  | 0.307057  |     0.201252 |
| Error c.r. Mínimo |  0.0373831 | 0.0196273 |     0.05534  |
| Error c.r. Óptimo |  1.16795   | 1.59538   |     1.18843  |
